In [1]:
import pandas as pd
import torch

# Below is a dataset consisting of four samples and the logits results of seven epoch runs of a binary classifier algorithm.

## You are asked to make calculations and then answer questions about the data.
##

### Note: The cells contain sample outputs, but they are not the correct outputs, they are displayed just as a guideline.

In [ ]:
s = {0: {'y_actual': 0, 'y_logits_epoch0': 20.94444, 'y_logits_epoch1': 0.40547, 'y_logits_epoch2': 0.40547, 'y_logits_epoch3': -0.40547, 'y_logits_epoch4': -0.40547, 'y_logits_epoch5': -1.38629, 'y_logits_epoch6': -3.89182}, 1: {'y_actual': 1, 'y_logits_epoch0': -20.19722, 'y_logits_epoch1': -0.40547, 'y_logits_epoch2': -0.40547, 'y_logits_epoch3': 0.40547, 'y_logits_epoch4': 0.40547, 'y_logits_epoch5': 1.38629, 'y_logits_epoch6': 4.59512}, 2: {'y_actual': 0, 'y_logits_epoch0': 1.38629, 'y_logits_epoch1': 0.04001, 'y_logits_epoch2': -0.04001, 'y_logits_epoch3': 0.04001, 'y_logits_epoch4': -0.04001, 'y_logits_epoch5': -0.8473, 'y_logits_epoch6': -3.17805}, 3: {'y_actual': 1, 'y_logits_epoch0': -0.20067, 'y_logits_epoch1': -0.04001, 'y_logits_epoch2': 0.04001, 'y_logits_epoch3': 0.04001, 'y_logits_epoch4': 0.04001, 'y_logits_epoch5': 1.09861, 'y_logits_epoch6': 3.89182}}

df = pd.DataFrame(s).T
y_actual = df['y_actual']
df = df.drop(columns='y_actual')

In [3]:
display(df)

,y_logits_epoch0,y_logits_epoch1,y_logits_epoch2,y_logits_epoch3,y_logits_epoch4,y_logits_epoch5,y_logits_epoch6
0,20.94444,0.40547,0.40547,-0.40547,-0.40547,-1.38629,-3.89182
1,-20.19722,-0.40547,-0.40547,0.40547,0.40547,1.38629,4.59512
2,1.38629,0.04001,-0.04001,0.04001,-0.04001,-0.84730,-3.17805
3,-0.20067,-0.04001,0.04001,0.04001,0.04001,1.09861,3.89182


# Q1: For each epoch, convert the logits results into standardized probabilities.
### The result is a new dataframe **df_probs** of the form below.
### Make sure to round the values to only 3 decimals.

In [4]:
# Q1: Convert logits to probabilities using PyTorch sigmoid
df_probs = df.copy()

# Apply sigmoid to each epoch column
for col in df.columns:
    logits_tensor = torch.tensor(df[col].values, dtype=torch.float32)
    probs_tensor = torch.sigmoid(logits_tensor)
    df_probs[col] = probs_tensor.numpy()

# Rename columns from 'y_logits_epochX' to 'y_probs_epochX'
df_probs.columns = [col.replace('logits', 'probs') for col in df_probs.columns]

# Round to 3 decimal places
df_probs = df_probs.round(3)

# Display the probabilities dataframe
display(df_probs)

,y_probs_epoch0,y_probs_epoch1,y_probs_epoch2,y_probs_epoch3,y_probs_epoch4,y_probs_epoch5,y_probs_epoch6
0,1.00,0.60,0.60,0.40,0.40,0.20,0.02
1,0.00,0.40,0.40,0.60,0.60,0.80,0.99
2,0.80,0.51,0.49,0.51,0.49,0.30,0.04
3,0.45,0.49,0.51,0.51,0.51,0.75,0.98


# Now, for each of the epochs, calculate and print:

### 1.   The predictions based on the logits/prob
### 2.   The accuracy of the predictions
### 3   The classification loss of the results (reminder: BCE = Binary Cross Entropy, also called "Log Loss")


In [5]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

# Q2: For each epoch, calculate predictions, accuracy, and BCE loss
for col in df_probs.columns:
    # Get probabilities as tensor
    probs_tensor = torch.tensor(df_probs[col].values, dtype=torch.float32)
    y_actual_tensor = torch.tensor(y_actual.values, dtype=torch.float32)

    # 1. Predictions (threshold at 0.5)
    predictions = (probs_tensor >= 0.5).int().numpy()

    # 2. Accuracy using scikit-learn
    accuracy = accuracy_score(y_actual, predictions)
    accuracy_pct = accuracy * 100

    # 3. BCE Loss using PyTorch
    loss = F.binary_cross_entropy(probs_tensor, y_actual_tensor, reduction='mean')

    # 4. Display results
    epoch_name = col.replace('y_probs_', '')
    print(f"y_logits_{epoch_name}")
    print(f"  Accuracy: {accuracy_pct:.1f}%")
    print(f"  Loss: {loss.item()}")


y_logits_epoch0
  Accuracy: 0.0%
  Loss: 50.601985931396484
y_logits_epoch1
  Accuracy: 0.0%
  Loss: 0.8148202896118164
y_logits_epoch2
  Accuracy: 50.0%
  Loss: 0.7948176860809326
y_logits_epoch3
  Accuracy: 75.0%
  Loss: 0.6020864248275757
y_logits_epoch4
  Accuracy: 100.0%
  Loss: 0.5920851230621338
y_logits_epoch5
  Accuracy: 100.0%
  Loss: 0.272661030292511
y_logits_epoch6
  Accuracy: 100.0%
  Loss: 0.022819427773356438


# Questions for understanding the relationships between accuracy and loss:



*   Both epoch 0 and epoch 1 have the same accuracy. Why do their losses differ so much?
*   Conversely, for epoch 1 and epoch 2 - why do their losses differ so little even though their accuracy is so improved?
* Why is the loss for Epoch 6 so low, even though it is exactly the same accuracy as epoch 4&5 ? What made it so much "better"?




---



# ANSWERS TO Q3: Understanding the Relationship Between Accuracy and Loss

## Q3a: Both epoch 0 and epoch 1 have the same accuracy. Why do their losses differ so much?

**Answer:**

The key insight is that **loss penalizes confidence in wrong predictions**, not just being wrong.

Let's examine the data more closely:


In [6]:

# Create a comparison dataframe
comparison = pd.DataFrame({
    'y_actual': y_actual,
    'epoch0_logits': df['y_logits_epoch0'],
    'epoch0_probs': df_probs['y_probs_epoch0'],
    'epoch1_logits': df['y_logits_epoch1'],
    'epoch1_probs': df_probs['y_probs_epoch1']
})
display(comparison)


,y_actual,epoch0_logits,epoch0_probs,epoch1_logits,epoch1_probs
0,0.0,20.94444,1.00,0.40547,0.60
1,1.0,-20.19722,0.00,-0.40547,0.40
2,0.0,1.38629,0.80,0.04001,0.51
3,1.0,-0.20067,0.45,-0.04001,0.49



**Explanation:**

Even if two epochs have the same accuracy (same number of correct/incorrect predictions), their losses can differ dramatically based on **how confident** the model was in its predictions:

- **Epoch 0** likely has **extreme logits** (very large positive or negative values), leading to probabilities very close to 0 or 1
- When the model is **overconfident and wrong** (e.g., predicting p=0.99 when y=0, or p=0.01 when y=1), the BCE loss becomes very large
- The loss function: BCE = -[y·log(p) + (1-y)·log(1-p)] approaches infinity as p approaches 0 when y=1, or as p approaches 1 when y=0

- **Epoch 1** has **moderate logits**, leading to probabilities closer to 0.5
- Even when wrong, the model is **less confident**, so the penalty (loss) is much smaller

**Key Takeaway:** Loss measures not just correctness but also **confidence**. A model that is wrong with high confidence is penalized more heavily than a model that is wrong but uncertain.

---

## Q3b: Epoch 1 and epoch 2 have different accuracy, but why do their losses differ so little?

**Answer:**

This demonstrates that **accuracy improvements near the decision boundary (p≈0.5) contribute minimally to loss reduction**.


In [7]:

# Compare epochs 1 and 2
comparison_12 = pd.DataFrame({
    'y_actual': y_actual,
    'epoch1_probs': df_probs['y_probs_epoch1'],
    'epoch1_pred': (df_probs['y_probs_epoch1'] >= 0.5).astype(int),
    'epoch2_probs': df_probs['y_probs_epoch2'],
    'epoch2_pred': (df_probs['y_probs_epoch2'] >= 0.5).astype(int)
})
display(comparison_12)


,y_actual,epoch1_probs,epoch1_pred,epoch2_probs,epoch2_pred
0,0.0,0.60,1,0.60,1
1,1.0,0.40,0,0.40,0
2,0.0,0.51,1,0.49,0
3,1.0,0.49,0,0.51,1


**Explanation:**

- **Epoch 2 has better accuracy** because some predictions crossed the 0.5 threshold compared to Epoch 1
- However, if these predictions are **barely correct** (e.g., p=0.51 for y=1, or p=0.49 for y=0), they contribute very little to reducing the loss
- The BCE loss changes gradually near p=0.5, so moving from p=0.49 (incorrect) to p=0.51 (correct) changes accuracy by 100% but changes loss only slightly

**Key Takeaway:** A prediction can be technically "correct" (crosses the 0.5 threshold) but still highly uncertain. Loss rewards confident correct predictions much more than borderline correct predictions. This is why loss is a better optimization target than accuracy - it encourages the model to be more confident in correct predictions.

---

## Q3c: Why is the loss for Epoch 6 so low, even though it has exactly the same accuracy as epochs 3, 4, and 5?

**Answer:**

This demonstrates the most important lesson: **loss rewards confidence in correct predictions**.


In [8]:


# Compare accuracy-matched epochs
comparison_final = pd.DataFrame({
    'y_actual': y_actual,
    'epoch3_probs': df_probs['y_probs_epoch3'],
    'epoch4_probs': df_probs['y_probs_epoch4'],
    'epoch5_probs': df_probs['y_probs_epoch5'],
    'epoch6_probs': df_probs['y_probs_epoch6'],
    'epoch6_pred': (df_probs['y_probs_epoch6'] >= 0.5).astype(int)
})
display(comparison_final)

# Calculate loss contributions per sample for epoch 6
y_tensor = torch.tensor(y_actual.values, dtype=torch.float32)
p6_tensor = torch.tensor(df_probs['y_probs_epoch6'].values, dtype=torch.float32)
loss_per_sample = -(y_tensor * torch.log(p6_tensor) + (1 - y_tensor) * torch.log(1 - p6_tensor))

print("\nLoss contribution per sample for Epoch 6:")
for i, loss_val in enumerate(loss_per_sample):
    print(f"  Sample {i} (y={int(y_actual.iloc[i])}): {loss_val.item():.4f}")


,y_actual,epoch3_probs,epoch4_probs,epoch5_probs,epoch6_probs,epoch6_pred
0,0.0,0.40,0.40,0.20,0.02,0
1,1.0,0.60,0.60,0.80,0.99,1
2,0.0,0.51,0.49,0.30,0.04,0
3,1.0,0.51,0.51,0.75,0.98,1



Loss contribution per sample for Epoch 6:
  Sample 0 (y=0): 0.0202
  Sample 1 (y=1): 0.0101
  Sample 2 (y=0): 0.0408
  Sample 3 (y=1): 0.0202


**Explanation:**

All these epochs have the same accuracy (same samples correct/incorrect), but **Epoch 6 has much higher confidence in its correct predictions**:

- For samples where the model is **correct**, higher confidence (p closer to 0 or 1) → **lower loss**
  - For y=1: p=0.99 gives loss=-log(0.99)=0.01, while p=0.60 gives loss=-log(0.60)=0.51
  - For y=0: p=0.01 gives loss=-log(0.99)=0.01, while p=0.40 gives loss=-log(0.60)=0.51
  
- **Epoch 6's logits are more extreme** (larger magnitudes), meaning the model is more confident
- Since these confident predictions are **correct**, the loss is minimized

**Key Takeaway:** Good model training should not just flip predictions from wrong to right - it should also **increase confidence in correct predictions**. This is why we optimize loss, not accuracy: loss provides a smooth gradient that encourages both correctness AND confidence, making it ideal for gradient-based optimization algorithms.